In [1]:
!pip install tensorflow

In [2]:
!rm -rf /content/id_docs_resized
!rm -rf /content/my_dataset/

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
import os
import shutil

In [4]:
!apt-get install -y unrar
!unrar x /content/id_docs_resized_224.rar /content/

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/id_docs_resized_224.rar

Creating    /content/id_docs_resized                                  OK
Creating    /content/id_docs_resized/train                            OK
Creating    /content/id_docs_resized/train/driving_licence            OK
Extracting  /content/id_docs_resized/train/driving_licence/image_1.jpeg       0%  OK 
Extracting  /content/id_docs_resized/train/driving_licence/image_10.jpg       0%  OK 
Extracting  /content/id_docs_resized/train/driving_licence/image_11.jpg       0%  OK 
Extracting  /content/id_docs_resized/train/driving_licence/image_12.jpg       0%  OK 
Extracting  /content/id_docs_resized/train/driving_licen

In [35]:
DATASET_PATH = 'id_docs_resized'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_EPOCHS = 20
LEARNING_RATE = 0.0001
VALIDATION_SPLIT = 0.2  # Use 20% of training data for validation
CONFIDENCE_THRESHOLD = 0.86 # Set a confidence threshold 86%

In [36]:
if not os.path.exists(DATASET_PATH):
    print(f"Error: Dataset directory '{DATASET_PATH}' not found.")
    print("Please create the directory with a 'train' and 'validation' subfolder,")
    print("and place your images in category-specific subfolders.")
    exit()

In [37]:
# --- Data Augmentation and Loading ---
# We use data augmentation to create more training examples from a small dataset.
# This helps the model generalize better and reduces overfitting.
train_datagen = image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=VALIDATION_SPLIT
)

In [38]:
# Only rescale the validation data, no augmentation
validation_datagen = image.ImageDataGenerator(rescale=1./255)

In [39]:
# Load the training and validation data from the directory
print("Loading training data...")
train_generator = train_datagen.flow_from_directory(
    os.path.join(DATASET_PATH, 'train'),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

print("\nLoading validation data...")
validation_generator = train_datagen.flow_from_directory(
    os.path.join(DATASET_PATH, 'train'),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Loading training data...
Found 286 images belonging to 4 classes.

Loading validation data...
Found 70 images belonging to 4 classes.


In [40]:
# Get the number of classes from the generator
num_classes = train_generator.num_classes
class_names = list(train_generator.class_indices.keys())
print(f"\nFound {num_classes} classes: {class_names}")


Found 4 classes: ['driving_licence', 'new_nic', 'old_nic', 'passport']


In [41]:
# --- Build the Model with Transfer Learning ---
# Load the pre-trained MobileNetV2 model without the top classification layers.
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
)

# Freeze the convolutional base
base_model.trainable = False
print("\nBase MobileNetV2 model loaded and frozen.")

# Create the new model on top of the base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Reduces the spatial dimensions of the feature maps
    Dense(512, activation='relu'),
    Dropout(0.5), # Add dropout to prevent overfitting
    Dense(num_classes, activation='softmax') # The final classification layer
])


Base MobileNetV2 model loaded and frozen.


In [42]:
# --- Compile and Train the Model (Feature Extraction Phase) ---
print("\nCompiling model...")
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

print("\nTraining model...")
history = model.fit(
    train_generator,
    epochs=NUM_EPOCHS,
    validation_data=validation_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=validation_generator.samples // BATCH_SIZE
)


Compiling model...


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,915,908 (11.12 MB)

 Trainable params: 657,924 (2.51 MB)

 Non-trainable params: 2,257,984 (8.61 MB)


Training model...
Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.3608 - loss: 1.5256 - val_accuracy: 0.6875 - val_loss: 0.8092
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - accuracy: 0.5000 - loss: 1.0089 - val_accuracy: 0.6875 - val_loss: 0.7997
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 670ms/step - accuracy: 0.6493 - loss: 0.8926 - val_accuracy: 0.7656 - val_loss: 0.5519
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 170ms/step - accuracy: 0.8125 - loss: 0.6548 - val_accuracy: 0.7656 - val_loss: 0.5324
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 486ms/step - accuracy: 0.7438 - loss: 0.6601 - val_accuracy: 0.8906 - val_loss: 0.3529
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - accuracy: 0.8750 - loss: 0.3992 - val_accuracy: 0.8594 - val_loss: 0.3526
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 581ms/step - accuracy: 0.8499 - loss: 0.4315 - val_accuracy: 0.9375 - val_loss: 0.2549
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step - accuracy: 0.8750 - loss: 0.4939 - val_accuracy: 0

In [22]:
# --- Fine-Tuning the Model ---
# This is a critical step for small datasets. We unfreeze some top layers
# of the pre-trained model and jointly train them with the new classification
# layers. This allows the model to adapt the general features to your specific data.
print("\nUnfreezing and fine-tuning the model...")
base_model.trainable = True

# Fine-tune from a specific layer onwards. A good practice is to unfreeze
# from the top layers to avoid corrupting low-level features.
fine_tune_at = 140 # MobileNetV2 has 155 layers, we'll fine-tune the top ~15
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Re-compile the model with a very low learning rate for fine-tuning
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE / 10), # A much lower learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

print(f"\nTraining with fine-tuning for {NUM_EPOCHS} more epochs...")
history_fine_tune = model.fit(
    train_generator,
    epochs=NUM_EPOCHS,
    validation_data=validation_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=validation_generator.samples // BATCH_SIZE
)


Unfreezing and fine-tuning the model...


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,915,908 (11.12 MB)

 Trainable params: 1,697,924 (6.48 MB)

 Non-trainable params: 1,217,984 (4.65 MB)


Training with fine-tuning for 20 more epochs...
Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.8857 - loss: 0.4096 - val_accuracy: 1.0000 - val_loss: 0.0843
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 319ms/step - accuracy: 0.8438 - loss: 0.4621 - val_accuracy: 0.9688 - val_loss: 0.1311
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 742ms/step - accuracy: 0.9242 - loss: 0.3089 - val_accuracy: 0.9844 - val_loss: 0.0894
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 285ms/step - accuracy: 0.9333 - loss: 0.2532 - val_accuracy: 0.9688 - val_loss: 0.1268
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 672ms/step - accuracy: 0.9094 - loss: 0.3113 - val_accuracy: 1.0000 - val_loss: 0.0682
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step - accuracy: 0.8125 - loss: 0.4086 - val_accuracy: 0.9688 - val_loss: 0.1115
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 678ms/step - accuracy: 0.9097 - loss: 0.2653 - val_accuracy: 0.9844 - val_loss: 0.0860
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step - accuracy: 0.9688 - 

In [44]:
# --- Save the Model ---
model.save('image_classifier_transfer_learning.h5')
model.save('image_classifier_transfer_learning.keras' , save_format='keras')
print("\nModel saved to 'image_classifier_transfer_learning.h5'")



Model saved to 'image_classifier_transfer_learning.h5'


In [26]:

# --- Predict a new image (Example) ---
def predict_image(model, img_path):
    """
    Predicts the class of an image and checks the confidence against a threshold.
    """
    img = image.load_img(img_path, target_size=IMAGE_SIZE)
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    img_array = img_array / 255.0 # Rescale the pixel values

    print(img)

    predictions = model.predict(img_array)
    confidence = tf.reduce_max(predictions[0]) # Get the highest confidence score

    print(f"\nPrediction for {img_path}:")

    # If the highest confidence is below the threshold, classify as 'Unknown'
    if confidence < CONFIDENCE_THRESHOLD:
        print(f"Predicted class: Unknown / Invalid")
        print(f"Confidence: {confidence.numpy() * 100:.2f}% (Below threshold of {CONFIDENCE_THRESHOLD * 100:.2f}%)")
    else:
        predicted_class_index = tf.argmax(predictions[0])
        predicted_class = class_names[predicted_class_index]
        print(f"Predicted class: {predicted_class}")
        print(f"Confidence: {confidence.numpy() * 100:.2f}%")

In [34]:
predict_image(model, 'image_9.jpeg')

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7CB838DFF4D0>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step

Prediction for image_9.jpeg:
Predicted class: Unknown / Invalid
Confidence: 39.47% (Below threshold of 86.00%)
